In [1]:
#Decorators 101

def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [2]:
target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

In [4]:
#When Python executes decorators

registry = []
def register(func):
    print(f'running register({func}')
    registry.append(func)
    return func
@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()') #decorated

def main():
    print('running main()')
    print('registry->', registry)
    f1()
    f2()
    f3()
if __name__ == '__main__':
    main()

running register(<function f1 at 0x1052fdd00>
running register(<function f2 at 0x1052fe160>
running main()
registry-> [<function f1 at 0x1052fdd00>, <function f2 at 0x1052fe160>]
running f1()
running f2()
running f3()


In [5]:
#Closures
class Averager:
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_values):
        self.series.append(new_values)
        total = sum(self.series)
        return total / len(self.series)

In [6]:
avg = Averager()
avg(10)

10.0

In [7]:
avg(11)

10.5

In [8]:
def make_averager():
    series = [] #free variable
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [9]:
avg = make_averager()
avg(10)

10.0

In [10]:
avg(11)

10.5

In [11]:
avg.__code__.co_varnames

('new_value', 'total')

In [12]:
avg.__code__.co_freevars

('series',)

In [13]:
avg.__closure__

(<cell at 0x105249480: list object at 0x1052d4140>,)

In [15]:
#nonlocal Declaration

#broken example
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1 #makes it a local var
        total += new_value
        return total / count

    return averager

In [16]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [17]:
def  make_averager():
    count , total = 0, 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager()

In [18]:
#Implementing simple decorator

import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{name}] {elapsed:0.3f}s | {arg_str}')
        return result
    return clocked

In [19]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n  < 2 else n * factorial(n - 1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[snooze] 0.125s | 0.123
**************************************** Calling factorial(6)
[factorial] 0.000s | 1
[factorial] 0.000s | 2
[factorial] 0.000s | 3
[factorial] 0.000s | 4
[factorial] 0.000s | 5
[factorial] 0.000s | 6
6! = 720


In [21]:
#standard library decorators
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

In [22]:
clock.__name__

'clock'

In [26]:
#Decorators in standard library
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000025s] fibonacci(0) -> 0
[0.00000071s] fibonacci(1) -> 1
[0.00026904s] fibonacci(2) -> 1
[0.00000017s] fibonacci(1) -> 1
[0.00000021s] fibonacci(0) -> 0
[0.00000017s] fibonacci(1) -> 1
[0.00001988s] fibonacci(2) -> 1
[0.00002913s] fibonacci(3) -> 2
[0.00033533s] fibonacci(4) -> 3
[0.00000029s] fibonacci(1) -> 1
[0.00000017s] fibonacci(0) -> 0
[0.00000013s] fibonacci(1) -> 1
[0.00055163s] fibonacci(2) -> 1
[0.00056675s] fibonacci(3) -> 2
[0.00000017s] fibonacci(0) -> 0
[0.00000025s] fibonacci(1) -> 1
[0.00023546s] fibonacci(2) -> 1
[0.00000013s] fibonacci(1) -> 1
[0.00000013s] fibonacci(0) -> 0
[0.00000017s] fibonacci(1) -> 1
[0.00000883s] fibonacci(2) -> 1
[0.00001679s] fibonacci(3) -> 2
[0.00026033s] fibonacci(4) -> 3
[0.00083633s] fibonacci(5) -> 5
[0.00119292s] fibonacci(6) -> 8
8


In [28]:
@functools.cache #improve performance
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000042s] fibonacci(0) -> 0
[0.00000083s] fibonacci(1) -> 1
[0.00039829s] fibonacci(2) -> 1
[0.00000079s] fibonacci(3) -> 2
[0.00041487s] fibonacci(4) -> 3
[0.00000046s] fibonacci(5) -> 5
[0.00042929s] fibonacci(6) -> 8
8


In [29]:
#Parametrized Decorators
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry->', registry)
f1()


running register(<function f1 at 0x1052fd9e0>)
running main()
registry-> [<function f1 at 0x1052fd9e0>]
running f1()


In [30]:
registry = set()

def register(active=True):
    def decorate(func):
        print(f'running register({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate


In [31]:
@register(active = False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
registry

running register(<function f1 at 0x107225120>)
running register(<function f2 at 0x105313420>)


{<function __main__.f2()>}

In [32]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class Clock:  

    def __init__(self, fmt=DEFAULT_FMT):  
        self.fmt = fmt

    def __call__(self, func):  
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)  
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked